In [1]:
import psycopg2
import pandas as pd
import pyodbc

In [2]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                        "Server=192.168.5.18\CROPNUT;"
                        "Database=cropnuts;"
                        "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
averaged_spectra = pd.read_csv("outputFiles/averaged_spectra_samples.csv")

In [4]:
averaged_spectra

,Unnamed: 0,metadata_id,averaged_spectra,sample_code
0,0,31,"{'522': [2.3244656573818396, 2.300363936880086...",CO064SA0543
1,1,17,"{'522': [2.3243741634120374, 2.307540485180485...",CS006SA0042
2,2,19,"{'522': [2.2630152586026915, 2.290110779246487...",FA419-51SA0016
3,3,11,"{'522': [2.154119577513004, 2.1818486646526885...",FA419-43SA0001
4,4,12,"{'522': [2.163048776013803, 2.154486397031147,...",FA419-46SA0003
...,...,...,...,...
5451,5451,6496,"{'522': [2.1780593727169077, 2.237326539442723...",CC176SA1451
5452,5452,6497,"{'522': [2.1004350637315294, 2.081424588592166...",CC176SA0688
5453,5453,6498,"{'522': [1.9198362844262806, 1.919189491471532...",CC176SA2142
5454,5454,6499,"{'522': [2.0913684528685414, 2.179166172188140...",CC176SA2517


In [5]:
sample_codes = averaged_spectra['sample_code'].values

In [6]:
lims = pd.read_sql(f"SELECT sample_code,chemical_name,result FROM SampleResults WHERE sample_code IN {tuple(sample_codes)}", con=conn_lims)

C:\Users\Tsuma Thomas\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
lims = lims[['sample_code','chemical_name','result']]

In [8]:
lims

,sample_code,chemical_name,result
0,CH006SA5782,None,None
1,CH006SA5438,None,None
2,CM093SA0305,None,None
3,CV062SA0344,None,None
4,CH006SA5526,None,None
...,...,...,...
131045,CA602SA0023,None,None
131046,CA602SA0024,None,None
131047,CA602SA0024,None,None
131048,CA602SA0024,None,None


In [9]:
lims = pd.pivot_table(lims, values='result', index=['sample_code'], columns=['chemical_name'], aggfunc='max')

In [10]:
chemicals = ['aluminium', 
            'phosphorus', 'ph', 'exchangeable_acidity', 'calcium', 'magnesium',
              'sulphur', 'sodium', 'iron', 'manganese', 'boron', 'copper', 'zinc', 'potassium',
             'ec_salts', 'cec', 'sand', 'silt', 'clay']

In [11]:
for column in lims.columns:
    col = column.lower()
    col = col.replace("(","").replace(")","")
    col = col.split(" ")
    col = "_".join(col)
    lims = lims.rename(columns={column:col})
for column in lims.columns:
    if column not in chemicals:
        lims.drop(column, axis=1, inplace=True)

In [12]:
(lims.columns)
# /rEMOVE ORGANICCARBON AND TOTALNITROGEN

Index(['aluminium', 'boron', 'calcium', 'clay', 'copper', 'ec_salts',
       'exchangeable_acidity', 'iron', 'magnesium', 'manganese', 'phosphorus',
       'potassium', 'sand', 'silt', 'sodium', 'sulphur', 'zinc', 'ph'],
      dtype='object', name='chemical_name')

In [13]:
lims.to_csv("outputFiles/joined_spectra_with_lims.csv")